<a href="https://colab.research.google.com/github/attackmood/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_8%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오픈소스 기반 데이터 분석 8강

# 데이터 분석 방법론

## 8-1 통계적 모형을 활용한 분석 (statsmodels)

In [6]:
import pandas as pd
import random
# statsmodels의 formula api 임포트
import statsmodels.formula.api as smf

## 난수 시드 설정
random.seed(1)

## 데이터 생성
X = list(range(1, 11))
y = [2*x + 1 + random.gauss(0, 1) for x in X] # x의 값에 가우시안값을 적용해서 y값을 만듦

## 데이터프레임 생성
data = pd.DataFrame({'X': X, 'y': y})

## OLS(최소 제곱법) 회귀 모델 생성 및 학습: 잔차의 제곱의 합이 가장 작은 선을 찾음
## 'y'를 종속 변수로 하고 다른 특성(X)을 독립 변수로 사용
model = smf.ols(formula='y~ X', data=data).fit()
# 'y ~ X'는 R 스타일의 모델 공식. "y는 X에 의해 설명된다"

## 모델 요약 결과 출력
print(model.summary())


# model.summary()가 보여주는 주요 지표들:

# R-squared: 모델이 데이터의 변동을 얼마나 설명하는지 (0~1 사이). 1에 가까울수록 좋다
# Coefficients: 추정된 절편(Intercept)과 기울기(X의 계수)
# p-value: 해당 변수가 통계적으로 유의미한지 판단 (보통 0.05 미만이면 유의미)
# Standard Error: 추정값의 불확실성 정도

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                     345.0
Date:                Fri, 19 Dec 2025   Prob (F-statistic):           7.28e-08
Time:                        06:43:57   Log-Likelihood:                -12.020
No. Observations:                  10   AIC:                             28.04
Df Residuals:                       8   BIC:                             28.65
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7622      0.615      2.866      0.0

## 8-2 기계학습을 활용한 분석 (scikit-learn)

In [14]:
# scikit-learn의 앙상블(ensemble) 알고리즘 중 RandomForestClassifier 임포트
from sklearn.ensemble import RandomForestClassifier
# 모델 평가를 위한 데이터 분할(split) 함수 임포트
from sklearn.model_selection import train_test_split
# 모델의 정확도(accuracy)를 측정하는 함수 임포트
from sklearn.metrics import accuracy_score
# 예제 데이터셋(유방암 데이터) 로드 함수 임포트
from sklearn.datasets import load_breast_cancer


## 데이터 로드 및 분할
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

## 학습/테스트 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 랜덤 포레스트 모델 생성 및 학습
model = RandomForestClassifier(n_estimators=100) # 100개의 서로 다른 트리를 만듦 -> 각 트리가 독립적으로 예측 -> 다수결로 최종 결정
model.fit(X_train, y_train)

## 예측 및 정확도 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

# 각각의 환자들에 대해 cancer가 있는지 없는지 정확도 측정
# 랜덤포레스트: 예측할 때 복수의 사람들한테 물어봐서 정확도를 예측한다?
# 100개의 결정 트리(decision tree)를 만듭니다 (n_estimators=100)
# 각 트리는 데이터의 일부만 보고 학습합니다
# 예측할 때 100개 트리가 투표합니다

# 트리1: 악성, 트리2: 양성, 트리3: 악성, ...
# 과반수가 선택한 것이 최종 예측

# 앙상블 학습
# - 각 트리가 다른 실수를 하면, 다수결로 오류를 상쇄
# - 과적합(overfitting)을 방지

0.9707602339181286


## 8-3 딥러닝을 활용한 분석 (tensorflow)

In [13]:
from sklearn.metrics import classification_report
# 딥러닝 분석을 위한 tensorflow 임포트
import tensorflow as tf

## CIFAR-10 데이터셋 로드 및 전처리(이미지 분류 문제)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

## MobileNetV2 기반 모델 구축
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3)) # 전이학습
base_model.trainable = False # 사전 학습된 부분은 **동결(freeze)**해서 가중치를 바꾸지 않음
model = tf.keras.Sequential([
    tf.keras.layers.Resizing(160, 160),         # 1. 이미지 크기 조정
    base_model,                                 # 2. 특성 추출기: 이미지에서 의미있는 패턴 추출
    tf.keras.layers.GlobalAveragePooling2D(),   # 3. 차원 축소: 추출된 특성맵을 1차원 벡터로 압축
    tf.keras.layers.Dense(10)                   # 4. 분류기: 10개 클래스에 대한 점수 출력
])

## 모델 컴파일
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 모델 학습
model.fit(x_train, y_train, epochs=3, validation_split=0.2)

## 예측 및 성능 평가
y_pred = tf.argmax(model.predict(x_test), axis=1)
print("\nTest Accuracy:", model.evaluate(x_test, y_test, verbose=0)[1])
print("\nClassification Report:\n", classification_report(y_test, y_pred))

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 54s 30ms/step - accuracy: 0.6917 - loss: 0.9039 - val_accuracy: 0.8050 - val_loss: 0.5641
Epoch 2/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.8209 - loss: 0.5139 - val_accuracy: 0.8208 - val_loss: 0.5177
Epoch 3/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.8375 - loss: 0.4729 - val_accuracy: 0.8208 - val_loss: 0.5254
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step

Test Accuracy: 0.8180000185966492

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.88      0.83      1000
           1       0.94      0.87      0.91      1000
           2       0.81      0.72      0.76      1000
           3       0.77      0.59      0.67      1000
           4       0.74      0.80      0.77      1000
           5       0.68      0.84      0.75      1000
           6       0.8

## 8-4 EDA 예시

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes

## 당뇨병 데이터셋 로드 및 데이터프레임 생성
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target

## dataframe의 describe를 이용하여 기본 통계량 출력

## 상관 행렬 시각화
plt.figure(figsize=(12,8))
# dataframe의 각 열(변수)들 간의 상관 계수(correlation coefficient)를 계산하고 corr_matrix로 저장
# seaborn으로 corr_matrix를 heatmap으로 시각화
plt.title('Diabetes Dataset Correlation Matrix')
plt.show()

## 8-5 기술통계량 분석

In [ ]:
import pandas as pd
import seaborn as sns

## 아이리스 데이터셋 로드
df = sns.load_dataset('iris')

# 기술 통계량
print("=== 기본 통계량 ===")
print(df.describe())

# 데이터 미리보기
print("\n=== 데이터 미리보기 ===")

# 그룹별 통계정보
print("\n=== 품종별 평균값 ===")

## 8-6 데이터 시각화 분석

In [ ]:
# 한글 처리를 위한 matplotlib 설정 (1)

!sudo apt-get install -y fonts-nanum
!sudo fc-cache –fv
!rm ~/.cache/matplotlib -rf

- 리소스 재시작

In [ ]:
# 한글 처리를 위한 matplotlib 설정 (2)

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## 아이리스 데이터셋 로드
df = sns.load_dataset('iris')

## 서브플롯 설정
plt.figure(figsize=(9, 12))
plt.suptitle('Iris 데이터셋 시각화', y=1.02, fontsize=14)

## 첫 번째 서브플롯: 꽃잎 길이 분포 (히스토그램)
plt.subplot(4, 1, 1)
# seaborn을 이용하여 petal_length를 10개 구간으로 나누어 히스토그램 표시
plt.title('꽃잎 길이 분포')
plt.xlabel('꽃잎 길이 (cm)')

## 두 번째 서브플롯: 꽃잎 길이 vs 너비 (산점도)
plt.subplot(4, 1, 2)
species_list = df['species'].unique()
colors = {'setosa':'red', 'versicolor':'green', 'virginica':'blue'}

for species in species_list:
    species_data = df[df['species'] == species]
    # matplotlib를 이용하여 길이, 넓이를 산점도로 표시
plt.title('꽃잎 길이 vs 너비')
plt.xlabel('꽃잎 길이 (cm)')
plt.ylabel('꽃잎 너비 (cm)')
plt.legend()

## 세 번째 서브플롯: 품종별 꽃받침 길이 (박스플롯)
plt.subplot(4, 1, 3)
# seaborn을 이용하여 품종 별 꽃받침 길이를 박스플롯으로 표시
plt.title('품종별 꽃받침 길이 비교')
plt.xlabel('품종')
plt.ylabel('꽃받침 길이 (cm)')

## 네 번째 서브플롯: 숫자형 특성 간 상관관계 (히트맵)
plt.subplot(4, 1, 4)
numeric_df = df.select_dtypes(include='number')
corr = numeric_df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('숫자형 특성 간 상관관계')

## 레이아웃 조정 및 그래프 표시
plt.tight_layout()
plt.show()

## 8-7 자동화된 EDA 도구

In [ ]:
!pip install ydata_profiling

In [ ]:
import seaborn as sns
from ydata_profiling import ProfileReport

## 아이리스 데이터셋 로드
df = sns.load_dataset('iris')

## 데이터 프로파일링 보고서 생성

## HTML 파일로 보고서 저장

## 보고서 표시 (Jupyter Notebook 환경에서 보고서가 바로 표시됨)